### Naive-Bayes Model

In [1]:
import numpy as np
import pandas as pd

np.set_printoptions(precision=4, suppress=True)

In [132]:
np.random.seed(4)

data = np.random.randn(60,5)
data = np.round(data, decimals=0)
X_train = data[:50, :-1]
y_train = data[:50, -1]

X_test = data[50:, :-1]
y_test = data[50:, -1]

In [133]:
X_test.shape, y_test.shape

((10, 4), (10,))

Base Model

In [134]:
from sklearn.naive_bayes import BernoulliNB
gnb = BernoulliNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [135]:
np.sum(y_pred == y_test) / len(y_test)

0.4

In [79]:
class Naive_Bayes:
    """Implements Naive-Bayes with Laplace Smoothing"""
    def compute_Px1y1(self, X, y):
        d = X.shape[1]
        return (np.sum(X[y==1] == 1) + 1) / (np.sum(y==1) + d)
    
    def compute_Px1y0(self, X, y):
        d = X.shape[1]
        return (np.sum(X[y==0] == 1) + 1) / (np.sum(y==0) + d)
    
    def compute_Px0y0(self, X, y):
        d = X.shape[1]
        return (np.sum(X[y==0] == 0) + 1) / (np.sum(y==0) + d)
    
    def compute_Px0y1(self, X, y):
        d = X.shape[1]
        return (np.sum(X[y==1] == 0) + 1) / (np.sum(y==1) + d)
    
    def compute_Py1(self, y):
        n = len(y)
        return (1/n) * np.sum(y==1)
    
    def compute_Py0(self, y):
        n = len(y)
        return (1/n) * np.sum(y==0)
    
    #def compute_Pxy(self, X, idx):
        #return np.power(self.phi, 
    
    def fit(self, X, y):
        d = X.shape[1]
        PhiXy_dict = dict()
        for idx in range(d):
            PhiXy_dict[idx] = dict()
            Phix1y1 = self.compute_Px1y1(X[:, idx], y)
            Phix1y0 = self.compute_Px1y0(X[:, idx], y)
            Phix0y0 =  self.compute_Px0y0(X[:, idx], y)
            Phix0y1 =  self.compute_Px0y1(X[:, idx], y)
            PhiXy_dict[idx].update({
                            'Phix0y0':Phix0y0, 
                            'Phix0y1':Phix0y1,
                            'Phix1y0':Phix1y0,
                            'Phix1y1':Phix1y1
                            })
        print(PhiXy_dict)
        self.phiXy = PhiXy_dict                  
        self.Phiy1 = self.compute_Py1(y)
        self.Phiy0 = self.compute_Py0(y)

    def predict_proba(self, X):
        Pxy0 = np.zeros(X.shape)
        Pxy1 = np.zeros(X.shape)
        n, d = X.shape
        
        for row in range(n):
            for col in range(d):
                if X[row, col] == 1:
                    Pxy0[row, col] = self.phiXy[col]['Phix1y0']
                    Pxy1[row, col] = self.phiXy[col]['Phix1y1']
                else:
                    Pxy0[row, col] = self.phiXy[col]['Phix0y0']
                    Pxy1[row, col] = self.phiXy[col]['Phix0y1']
        
        Pxy1 = np.multiply.reduce(Pxy1, axis=1)
        Pxy0 = np.multiply.reduce(Pxy0, axis=1)
        
        Py0 = Pxy0 * self.Phiy0
        Py0 = Py0.reshape(-1, 1)
        
        Py1 = Pxy1 * self.Phiy1
        Py1 = Py1.reshape(-1, 1)
                        
        return np.concatenate((Py0, Py1), axis=1)
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return np.argmax(proba, axis=1)

In [149]:
class Naive_Bayes:
    """Implements Naive-Bayes with Laplace Smoothing"""
    def compute_Px1y1(self, X, y):
        return (np.sum(X[y==1] == 1) + 1) / (np.sum(y==1) + self.d)
    
    def compute_Px1y0(self, X, y):
        return (np.sum(X[y==0] == 1) + 1) / (np.sum(y==0) + self.d)
    
    def compute_Px0y0(self, X, y):
        return (np.sum(X[y==0] == 0) + 1) / (np.sum(y==0) + self.d)
    
    def compute_Px0y1(self, X, y):
        return (np.sum(X[y==1] == 0) + 1) / (np.sum(y==1) + self.d)
    
    def compute_Py1(self, y):
        n = len(y)
        return (1/n) * np.sum(y==1)
    
    def compute_Py0(self, y):
        n = len(y)
        return (1/n) * np.sum(y==0)
    
    #def compute_Pxy(self, X, idx):
        #return np.power(self.phi, 
    
    def fit(self, X, y):
        self.d = X.shape[1]
        PhiXy_arr = []
        for idx in range(self.d):
            PhiXy = np.zeros((2,2))
            PhiXy[1,1] = self.compute_Px1y1(X[:, idx], y)
            PhiXy[1,0] = self.compute_Px1y0(X[:, idx], y)
            PhiXy[0,0] =  self.compute_Px0y0(X[:, idx], y)
            PhiXy[0,1] =  self.compute_Px0y1(X[:, idx], y)
            
            PhiXy_arr.append(PhiXy)
        PhiXy_arr = np.array(PhiXy_arr)
        #print(PhiXy_arr)
        self.phiXy = PhiXy_arr                  
        self.Phiy1 = self.compute_Py1(y)
        self.Phiy0 = self.compute_Py0(y)

    def predict_proba(self, X):
        Pxy0 = np.zeros(X.shape)
        Pxy1 = np.zeros(X.shape)
        n, d = X.shape
        
        for row in range(n):
            for col in range(d):
                if X[row, col] == 1:
                    Pxy0[row, col] = self.phiXy[col][1][0]
                    Pxy1[row, col] = self.phiXy[col][1][1]
                else:
                    Pxy0[row, col] = self.phiXy[col][0][0]
                    Pxy1[row, col] = self.phiXy[col][0][1]
        
        Pxy1_i = np.multiply.reduce(Pxy1, axis=1)
        Pxy0_i = np.multiply.reduce(Pxy0, axis=1)
        
        Py0 = Pxy0_i * self.Phiy0
        Py0 = Py0.reshape(-1, 1)
        
        Py1 = Pxy1_i * self.Phiy1
        Py1 = Py1.reshape(-1, 1)
                        
        return np.concatenate((Py0, Py1), axis=1)
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return np.argmax(proba, axis=1)

In [150]:
nb = Naive_Bayes()

In [151]:
nb.fit(X_train, y_train)

In [152]:
predictions = nb.predict(X_test)

In [153]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [154]:
np.sum(predictions == y_test) / len(y_test)

0.4

In [155]:
nb.predict_proba(X)[0:5]

array([[0.0028, 0.0027],
       [0.0061, 0.0034],
       [0.0024, 0.0068],
       [0.0031, 0.0009],
       [0.0024, 0.0068]])